In [3]:
import pandas as pd
import os
import numpy as np
import re
import matplotlib.pyplot as plt
from statsmodels.stats.anova import AnovaRM
from scipy import stats

make functions to grab subject info and also walk through all files within our data folder and look for behavioral data csv files

In [4]:
# this function gets the subject and run from the file name of the behavioral data file 
def extract_subject_and_run(input_str):
    # Define the regex patterns
    subject_pattern = re.compile(r'SF\d+')
    run_pattern = re.compile(r'run\d+')

    # Search for the patterns in the string
    subject_match = subject_pattern.search(input_str)
    run_match = run_pattern.search(input_str)

    # Extract the matched strings
    subject_number = subject_match.group(0) if subject_match else None
    run_number = run_match.group(0) if run_match else None
    
    return subject_number, run_number

#this function makes a list of our behavioral data filepaths
def find_csv_files(root_folder):
    file_paths = []
    
    # Check if root_folder exists
    if not os.path.exists(root_folder):
        print(f"Error: {root_folder} does not exist!")
        return file_paths
    
    # Walk through root_folder
    for foldername, subfolders, filenames in os.walk(root_folder):
        for file in filenames:
            if file.endswith('.csv') and not file.startswith('.'):  # Avoid hidden files
                file_path = os.path.join(foldername, file)
                # print(f"Appending CSV file: {file_path}")
                file_paths.append(file_path)
    
    return file_paths

use the functions we just made, and create a raw data dataframe

In [9]:
# Specify the main folder
main_folder = '/Users/jinjiang-macair/Library/CloudStorage/Box-Box/CNRISpring24_StabFlexTeam/Data'

# Find .csv files recursively
file_paths = find_csv_files(main_folder)

# initialize empty dataframe to store raw data
raw_data = pd.DataFrame()

for filePath in file_paths:
    df = pd.read_csv(filePath)

    # Extract subject and run information
    subject, run = extract_subject_and_run(filePath)
    df['subject'] = subject
    df['run'] = run
    
    # Append the raw data to the raw data dataframe
    raw_data = pd.concat([raw_data, df])

raw_data

,stim,ITI,cuedTask,taskSequence,congruency,corrResp,blockType,trials.thisRepN,trials.thisTrialN,trials.thisN,...,date,expName,psychopyVersion,frameRate,frameDur,Unnamed: 39,subject,thisRow.t,notes,Unnamed: 41
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2023-06-09_15h26.28.663,stability_flexibility_tradeoff,2022.2.5,59.962859,0.016667,NaN,SF05,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2023-06-09_15h26.28.663,stability_flexibility_tradeoff,2022.2.5,59.962859,0.016667,NaN,SF05,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2023-06-09_15h26.28.663,stability_flexibility_tradeoff,2022.2.5,59.962859,0.016667,NaN,SF05,NaN,NaN,NaN
3,1.0,1.5,p,n,c,1.0,C,0.0,0.0,0.0,...,2023-06-09_15h26.28.663,stability_flexibility_tradeoff,2022.2.5,59.962859,0.016667,NaN,SF05,NaN,NaN,NaN
4,8.0,1.0,p,r,c,2.0,C,0.0,1.0,1.0,...,2023-06-09_15h26.28.663,stability_flexibility_tradeoff,2022.2.5,59.962859,0.016667,NaN,SF05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,3.0,2.0,m,s,c,1.0,A,0.0,124.0,124.0,...,2024-02-06_13h27.38.199,stability_flexibility_tradeoff,2022.2.5,60.014031,0.016667,NaN,SF39,NaN,NaN,NaN
128,7.0,2.0,m,r,i,2.0,A,0.0,125.0,125.0,...,2024-02-06_13h27.38.199,stability_flexibility_tradeoff,2022.2.5,60.014031,0.016667,NaN,SF39,NaN,NaN,NaN
129,3.0,1.0,m,r,c,1.0,A,0.0,126.0,126.0,...,2024-02-06_13h27.38.199,stability_flexibility_tradeoff,2022.2.5,60.014031,0.016667,NaN,SF39,NaN,NaN,NaN
130,2.0,1.0,m,r,i,1.0,A,0.0,127.0,127.0,...,2024-02-06_13h27.38.199,stability_flexibility_tradeoff,2022.2.5,60.014031,0.016667,NaN,SF39,NaN,NaN,NaN


In [10]:
file_paths

['/Users/jinjiang-macair/Library/CloudStorage/Box-Box/CNRISpring24_StabFlexTeam/Data/20230609.SF05.SF05/Behavioral/SF05_stability_flexibility_tradeoff_run4_2023-06-09_15h26.28.663.csv',
 '/Users/jinjiang-macair/Library/CloudStorage/Box-Box/CNRISpring24_StabFlexTeam/Data/20230609.SF05.SF05/Behavioral/SF05_stability_flexibility_tradeoff_run3_2023-06-09_15h16.48.879.csv',
 '/Users/jinjiang-macair/Library/CloudStorage/Box-Box/CNRISpring24_StabFlexTeam/Data/20230609.SF05.SF05/Behavioral/SF05_stability_flexibility_tradeoff_run2_2023-06-09_15h05.35.381.csv',
 '/Users/jinjiang-macair/Library/CloudStorage/Box-Box/CNRISpring24_StabFlexTeam/Data/20230609.SF05.SF05/Behavioral/SF05_stability_flexibility_tradeoff_run1_2023-06-09_14h45.12.881.csv',
 '/Users/jinjiang-macair/Library/CloudStorage/Box-Box/CNRISpring24_StabFlexTeam/Data/20231006.SF21.SF21/Behavioral/SF21_stability_flexibility_tradeoff_run2_2023-10-06_14h09.04.530.csv',
 '/Users/jinjiang-macair/Library/CloudStorage/Box-Box/CNRISpring24_Sta

this is just to print out the column names

In [11]:
raw_data.columns.values.tolist()

['stim',
 'ITI',
 'cuedTask',
 'taskSequence',
 'congruency',
 'corrResp',
 'blockType',
 'trials.thisRepN',
 'trials.thisTrialN',
 'trials.thisN',
 'trials.thisIndex',
 'main2_start',
 'key_pressed',
 'key_RT',
 'routine_end',
 'scanner_start',
 'run',
 'fixation_start',
 'fixation_start_rel',
 'fixation_end_rel',
 'stimulus_start',
 'stimulus_start_rel',
 'stimulus_end_rel',
 'acc',
 'corrResp.1',
 'feedback_start',
 'feedback_start_rel',
 'feedback_end_rel',
 'blockType.1',
 'blockBreak_start',
 'blockBreak_start_rel',
 'blockBreak_end_rel',
 'Participant',
 'Run (1-4)',
 'date',
 'expName',
 'psychopyVersion',
 'frameRate',
 'frameDur',
 'Unnamed: 39',
 'subject',
 'thisRow.t',
 'notes',
 'Unnamed: 41']

now you get the average accuracy and reaction time for each subject :)  
here's the pandas documentation: https://pandas.pydata.org/pandas-docs/stable/index.html  
but also ask chatpgt and stackoverflow for help  
HINT: try the groupby function from pandas

get accuracy and rt

accuracy df is just all of the accurate trials

In [13]:
accuracy_df = (raw_data['acc'] == 1)

In [14]:
# Filter the rows where 'acc' is 1
correct_trials = raw_data[accuracy_df]

# Step 1: Overall average acc and key_RT for each subject
overall_avg_acc = raw_data.groupby('subject')['acc'].mean()
overall_avg_key_RT = correct_trials.groupby('subject')['key_RT'].mean() #only get RT on correct trials

# Convert each Series to a DataFrame
overall_avg_acc_df = overall_avg_acc.to_frame(name='acc')
overall_avg_key_RT_df = overall_avg_key_RT.to_frame(name='key_RT')

overall_avg = pd.concat([overall_avg_acc_df, overall_avg_key_RT_df], axis=1)

In [26]:
overall_avg

,acc,key_RT
subject,,
SF01,0.876953,0.775040
SF02,0.875000,0.917387
SF03,0.960938,0.769149
SF04,0.912109,0.873968
SF05,0.734375,0.924129
SF06,0.988281,0.814678
SF07,0.943359,0.893543
SF08,0.958984,0.819765
SF09,0.962891,0.780295


congruency effect

In [22]:
# Calculate the average RT for correct trials, grouped by subject and congruency
congruency_avg = correct_trials.groupby(['subject', 'congruency'])['key_RT'].mean()

# Unstack the congruency level of the index to get 'i' and 'c' as separate columns
congruency_avg_unstacked = congruency_avg.unstack()

In [23]:
congruency_avg

subject  congruency
SF01     c             0.725651
         i             0.833299
SF02     c             0.883324
         i             0.954634
SF03     c             0.695559
                         ...   
SF37     i             0.948813
SF39     c             0.959107
         i             0.934942
SF40     c             0.764731
         i             0.806893
Name: key_RT, Length: 76, dtype: float64

In [24]:
congruency_avg_unstacked

congruency,c,i
subject,,
SF01,0.725651,0.833299
SF02,0.883324,0.954634
SF03,0.695559,0.847050
SF04,0.833409,0.917219
SF05,0.882760,0.978777
SF06,0.775361,0.854622
SF07,0.836868,0.952859
SF08,0.770506,0.872558
SF09,0.730931,0.832758


In [28]:
 # Calculate the congruency effect as the difference between incongruent and congruent RTs
 # and add this column to our overall avg dataframe
overall_avg['congruency_effect'] = congruency_avg_unstacked['i'] - congruency_avg_unstacked['c']

# our overall avg dataframe now has a 'congruency_effect' column for each subject
overall_avg

,acc,key_RT,congruency_effect
subject,,,
SF01,0.876953,0.775040,0.107648
SF02,0.875000,0.917387,0.071311
SF03,0.960938,0.769149,0.151491
SF04,0.912109,0.873968,0.083810
SF05,0.734375,0.924129,0.096017
SF06,0.988281,0.814678,0.079261
SF07,0.943359,0.893543,0.115991
SF08,0.958984,0.819765,0.102053
SF09,0.962891,0.780295,0.101826


### get listwide proportion congruent effect
first get the avg rt for each block type for both incongruent and incongruent  

block proportions:  
A: 0.25 congruent, 0.25 switch.  
B: 0.25 congruent, 0.75 switch.  
C: 0.75 congruent, 0.25 switch.  
D: 0.75 congruent, 0.75 switch.  

In [37]:
# Step 3: LWPC effect
# Group the data and then unstack
congruency_and_block_type = correct_trials.groupby(['subject', 'blockType', 'congruency'])['key_RT'].mean()
congruency_and_block_type_unstacked = congruency_and_block_type.unstack(level=['blockType', 'congruency']) #this df has RT by block type and congruency, can use for plotting the block types against each other for congruency

In [38]:
congruency_and_block_type

subject  blockType  congruency
SF01     A          c             0.744833
                    i             0.812455
         B          c             0.825798
                    i             0.887126
         C          c             0.663771
                                    ...   
SF40     B          i             0.815193
         C          c             0.750663
                    i             0.791934
         D          c             0.773234
                    i             0.792327
Name: key_RT, Length: 304, dtype: float64

In [39]:
congruency_and_block_type_unstacked

blockType          A                   B                   C            \
congruency         c         i         c         i         c         i   
subject                                                                  
SF01        0.744833  0.812455  0.825798  0.887126  0.663771  0.747537   
SF02        0.895200  0.941414  0.916627  0.967559  0.845152  0.902734   
SF03        0.684137  0.837116  0.712594  0.821194  0.658829  0.873606   
SF04        0.828355  0.898364  0.929970  0.930391  0.795794  0.904905   
SF05        0.860924  0.980014  0.958929  0.996739  0.862242  0.944501   
SF06        0.849410  0.825586  0.772321  0.888333  0.770320  0.849756   
SF07        0.869958  0.953604  0.975295  0.961167  0.774597  0.935221   
SF08        0.821860  0.827629  0.841475  0.893882  0.748619  0.904992   
SF09        0.762968  0.812676  0.791063  0.860154  0.711162  0.777816   
SF10        0.802838  0.855362  0.905449  0.912434  0.820300  0.824532   
SF11        0.793387  0.913893  0.971952  1.027159  0.834688  1.004975   
SF12        0.834677  0.917829  0.832339  0.930832  0.811250  0.872861   
SF13        0.857477  0.872193  0.905149  0.951612  0.837222  0.950336   
SF14        0.758855  0.887970  0.893188  0.955493  0.766080  0.958027   
SF15        0.804244  0.875284  0.841485  0.945403  0.824159  1.001732   
SF16        0.831249  0.845452  0.836496  0.893386  0.816088  0.885310   
SF17        0.650928  0.552788  0.567712  0.577287  0.615722  0.572970   
SF18        0.781990  0.860988  0.967848  0.982714  0.802774  0.887138   
SF19        0.860465  0.932065  0.953983  1.011368  0.879990  0.920064   
SF20        0.926882  0.994065  0.990771  1.063600  0.871385  1.009965   
SF21        0.866893  0.905783  0.948880  1.082437  0.829002  0.884645   
SF22        0.863524  0.977924  0.893277  1.016839  0.819727  0.944119   
SF23        0.772840  0.818404  0.821019  0.871695  0.805173  0.769099   
SF24        0.934672  0.910938  0.973618  0.979702  0.781820  0.856970   
SF25        0.850755  0.913450  0.889668  0.945462  0.756762  1.005376   
SF26        0.777408  0.944633  0.754643  0.928592  0.683525  0.865075   
SF27        0.826674  0.944306  0.895732  0.970644  0.763109  0.926051   
SF28        0.652282  0.786679  0.757063  0.842283  0.658105  0.816103   
SF29        0.741124  0.765802  0.835744  0.836209  0.681434  0.751607   
SF30        0.858623  0.989789  0.985805  1.015530  0.900021  0.942656   
SF31        0.774299  0.827619  0.851567  0.938673  0.754182  0.918485   
SF32        0.908044  0.979147  0.999154  1.077897  0.945531  1.074136   
SF33        0.810350  0.856752  0.862971  0.884632  0.790184  0.831623   
SF34        0.644523  0.804634  0.632259  0.819445  0.649007  0.857791   
SF35        0.834898  0.945238  0.944992  1.027730  0.904504  1.052015   
SF37        0.833410  0.882521  0.875541  0.989743  0.796561  0.946724   
SF39        0.921120  0.884640  0.926384  0.960888  0.902943  0.968485   
SF40        0.750207  0.806297  0.789357  0.815193  0.750663  0.791934   

blockType          D            
congruency         c         i  
subject                         
SF01        0.746170  0.826215  
SF02        0.907385  1.017206  
SF03        0.729831  0.935863  
SF04        0.840577  0.952380  
SF05        0.888325  0.950173  
SF06        0.756701  0.844394  
SF07        0.841606  0.943581  
SF08        0.752165  0.916787  
SF09        0.721230  0.867585  
SF10        0.909195  0.994020  
SF11        0.797550  0.974646  
SF12        0.840289  0.860817  
SF13        0.946255  1.024896  
SF14        0.773148  0.925496  
SF15        0.863122  1.148950  
SF16        0.803831  0.877777  
SF17        0.527016  0.598258  
SF18        0.740645  0.965432  
SF19        0.887939  0.954110  
SF20        0.814104  1.043585  
SF21        0.930047  1.027152  
SF22        0.798775  0.946610  
SF23        0.887516  1.001635  
SF24        0.897544  0.935720  
SF25        0.764616  0.965086  
SF26        0.719291  0.932575  
SF27    

actually calculate lwpc effect

congruency effect in high congruency blocks - congruency effect in low congruency block  
1. blocks C and D are high congruency  
2. blocks A and B are low congruency  
3. So we can treat blocks C and D as one big high congruency block and blocks A and B as one big low congruency block  
4. Thus we can find the congruency effect for the combined high congruency block and subtract the congruency effect for the combined low congruency block  
(C_i + D_i - C_c - D_c) - (A_i + B_i - A_c - B_c)

In [42]:
incongruent_avg_high_con_blocks = congruency_and_block_type_unstacked['C']['i'] + congruency_and_block_type_unstacked['D']['i']
congruent_avg_high_con_blocks = congruency_and_block_type_unstacked['C']['c'] + congruency_and_block_type_unstacked['D']['c']
incongruent_avg_low_con_blocks = congruency_and_block_type_unstacked['A']['i'] + congruency_and_block_type_unstacked['B']['i']
congruent_avg_low_con_blocks = congruency_and_block_type_unstacked['A']['c'] + congruency_and_block_type_unstacked['B']['c']

overall_avg['lwpc_effect'] = (incongruent_avg_high_con_blocks - congruent_avg_high_con_blocks) - (incongruent_avg_low_con_blocks - congruent_avg_low_con_blocks)


In [43]:
overall_avg

,acc,key_RT,congruency_effect,lwpc_effect
subject,,,,
SF01,0.876953,0.775040,0.107648,0.034861
SF02,0.875000,0.917387,0.071311,0.070257
SF03,0.960938,0.769149,0.151491,0.159229
SF04,0.912109,0.873968,0.083810,0.150484
SF05,0.734375,0.924129,0.096017,-0.012793
SF06,0.988281,0.814678,0.079261,0.074941
SF07,0.943359,0.893543,0.115991,0.193081
SF08,0.958984,0.819765,0.102053,0.262820
SF09,0.962891,0.780295,0.101826,0.094210


In [45]:
# Select only numeric columns from the DataFrame (basically skip the subjects column cuz its all strings and cant take the average of strings)
overall_avg_numeric = overall_avg.select_dtypes(include=[np.number])

# Calculate the mean across subjects for each numeric column
means = overall_avg_numeric.mean()

# Calculate the standard error of the mean (SEM) for each numeric column
sem = overall_avg_numeric.sem()

print("Means:\n", means)
print("\nStandard Errors:\n", sem)

Means:
 acc                  0.876285
key_RT               0.854841
congruency_effect    0.096497
lwpc_effect          0.100008
dtype: float64

Standard Errors:
 acc                  0.017599
key_RT               0.012852
congruency_effect    0.008521
lwpc_effect          0.017255
dtype: float64
